## Combining bpmnconstraints with Declare4Py
To use bpmnconstraints with Declare4Py, first import the necessary modules

In [1]:
import os
from Declare4Py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("../../../", "tests", "test_logs","Sepsis Cases.xes.gz")
event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)
from Declare4Py.bpmnconstraints.bpmnconstraints import bpmnconstraints


/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/lark/utils.py:163: DeprecationWarning: module 'sre_parse' is deprecated
  import sre_parse
/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/lark/utils.py:164: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants
/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 3901.39it/s]
/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")
DEBUG:matplotlib:CACHEDIR=/home/fleph/.cache/matpl

Translate a bpmn model that you are interested in checking. For example, the nested and/or graph:
<figure>
    <img width=800" height="250" src="nested_and_or.png">
</figure>

In [2]:
from Declare4Py.ProcessModels.LTLModel import LTLModel
from Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer import LTLAnalyzer



path = "../examples/combinations/nested_and_or.xml"
model = bpmnconstraints(compile_path= path, constraint_type="LTLf")
# model now contains the translated graph
ltl_model = LTLModel()
for constraint in model:
    ltl_model.parse_from_string(content= constraint)
    analyzer = LTLAnalyzer(event_log, ltl_model)
    df = analyzer.run()
    print(f"{ltl_model.formula} is satisfiable? {ltl_model.check_satisfiability()}") 
    print("----------------------")
    print(ltl_model.parsed_formula)
    print(df)
    # This is currently wrong because of difference in ltlf dialect, but subject to change

100%|██████████| 5/5 [00:00<00:00, 177724.75it/s]


parallelgateway is satisfiable? True
----------------------
parallelgateway
     case:concept:name  accepted
0                    A     False
1                    B     False
2                    C     False
3                    D     False
4                    E     False
...                ...       ...
1045               HNA     False
1046               INA     False
1047               JNA     False
1048               KNA     False
1049               LNA     False

[1050 rows x 2 columns]
(F(task)) ->  (F(task)) is satisfiable? True
----------------------
(implies (eventually task) (eventually task))
     case:concept:name  accepted
0                    A      True
1                    B      True
2                    C      True
3                    D      True
4                    E      True
...                ...       ...
1045               HNA      True
1046               INA      True
1047               JNA      True
1048               KNA      True
1049               LNA    

Similary can be done with DECLARE

In [10]:
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser
from Declare4Py.ProcessModels.DeclareModel import DeclareModel


model = bpmnconstraints(compile_path= path)
declare_model = DeclareModel()
declare_model.parse_from_string("\n".join(model))
model_constraints = declare_model.get_decl_model_constraints()
model_act = declare_model.get_model_activities()
for idx, act in enumerate(model_act):
    print (idx, act)
for idx, constr in enumerate(model_constraints):
    print(idx, constr)


#basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=declare_model, consider_vacuity=False)
#conf_check_res: MPDeclareResultsBrowser = basic_checker.run()
#conf_check_res.get_metric(metric="num_activations")

100%|██████████| 5/5 [00:00<00:00, 71575.15it/s]


0 Init[parallelgateway] |
1 Co-Existence[task, task] |
2 Co-Existence[task, task] |
3 Co-Existence[task, task] |
4 Choice[task, task] |
